In [ ]:
import pandas as pd
import os, sys

# Filter all positive images out

In [ ]:
# Get all positive #1 (活动性出血) images -- 429 total images

In [ ]:
import os, sys, glob
pos_ind_df = pd.read_excel(r'C:\Users\wiqwe\Desktop\上交科研\c活动性出血.xlsx')
pos_ind_df.columns
name_ind = pos_ind_df[[' 姓名','出血典型病变图片号码']]

In [ ]:
# traverse all positive image folders
import re
folder_prefix = r'C:\Users\wiqwe\Desktop\上交科研\出血'
folders = glob.glob(os.path.join(folder_prefix,'*'))
# print(folders)
name_to_folder = {}

def name_extract(folder):
    pattern = re.compile('\d{6,10}(.+)')
    name = pattern.findall(folder)
    name_to_folder[name[0]] = folder
    return name

names = [name_extract(i) for i in folders]

In [ ]:
# Copy selected images from patient's folder to dataset folder
import shutil
pos_img_index = 0
POSITIVE_IMGS_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_positive_#1_imgs'
def func(a):
    name, img_inds = a.tolist()
    if name.endswith('2'): name = name[:-1]
#     print(name, img_inds,'|', type(img_inds), len(img_inds), 'called')
    img_inds = img_inds.replace(' ', '').replace('，', ',').strip(',').replace('.',',').split(',')
    name = name.strip()
#     print(type(img_inds), img_inds)
    
    def copy_file_func(img_ind):
        global pos_img_index
        full_path = os.path.join(name_to_folder[name], img_ind + '.jpg')
        print(full_path)
        paste_path = os.path.join(POSITIVE_IMGS_FOLDER, str(pos_img_index) + '.jpg')
        shutil.copyfile(full_path,paste_path)
        pos_img_index += 1
   
    _ = [copy_file_func(i) for i in img_inds]
#     print(name, img_inds)
    
_ = name_ind.apply(func, axis=1, raw=True)

In [ ]:
# Get all positive #2 (血管畸形) images -- 1169 total images
pos_ind_df = pd.read_excel(r'C:\Users\wiqwe\Desktop\上交科研\消化道出血病人表-血管畸形80.xlsx', dtype=str)
pos_ind_df.columns
name_ind = pos_ind_df[[' 姓名','病变图片编号']]

In [ ]:
import shutil
pos_img_index = 429
POSITIVE_IMGS_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_positive_#2_imgs'
def func(a):
    name, img_inds = a.tolist()
    if type(img_inds) != str: return
    print(name, img_inds, type(img_inds))
    img_inds = img_inds.replace(' ', '').replace('，', ',').strip(',').replace('.',',').split(',')
    name = name.strip()
    print(type(img_inds), img_inds)
    
    def copy_file_func(img_ind):
        global pos_img_index
        full_path = os.path.join(name_to_folder[name], img_ind + '.jpg')
        print(full_path)
        paste_path = os.path.join(POSITIVE_IMGS_FOLDER, str(pos_img_index) + '.jpg')
        shutil.copyfile(full_path,paste_path)
        pos_img_index += 1
   
    _ = [copy_file_func(i) for i in img_inds]
#     print(name, img_inds)
    
_ = name_ind.apply(func, axis=1, raw=True)

# Sample negative images

In [5]:
# Get negative images -- 454 * num_per_folder
import random
import glob, os, shutil
NEGATIVE_IMGS_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_negative_imgs'
folder_prefix = r'C:\Users\wiqwe\Desktop\上交科研\阴性'
neg_img_index = 0
num_per_folder = 5
folders = glob.glob(os.path.join(folder_prefix,'*'))
def copy_neg_image(image):
    global neg_img_index
    image_basename = os.path.basename(image)
    paste_path = os.path.join(NEGATIVE_IMGS_FOLDER, str(neg_img_index) + '.jpg')
    neg_img_index += 1
#     print(image)
    shutil.copyfile(image,paste_path)
    
for folder in folders:
    images = glob.glob(os.path.join(folder, '*'))
    if not images:
        continue
    elif len(images) > num_per_folder:
        images = random.sample(images, num_per_folder)
    for image in images:
        copy_neg_image(image)


# Train and validation split

In [41]:
# split train and validation set (copy files to different folder)
POSITIVE_1_IMGS_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_positive_#1_imgs'
POSITIVE_1_IMGS_TRAIN_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_positive_#1_imgs_train'
POSITIVE_1_IMGS_VALIDATION_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_positive_#1_imgs_validation'
POSITIVE_2_IMGS_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_positive_#2_imgs'
POSITIVE_2_IMGS_TRAIN_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_positive_#2_imgs_train'
POSITIVE_2_IMGS_VALIDATION_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_positive_#2_imgs_validation'
NEGATIVE_IMGS_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_negative_imgs'
NEGATIVE_IMGS_TRAIN_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_negative_imgs_train'
NEGATIVE_IMGS_VALIDATION_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_negative_imgs_validation'
VALIDATION_RATIO = 0.15

# split randomly
positive_1_imgs = glob.glob(os.path.join(POSITIVE_1_IMGS_FOLDER,'*'))
positive_1_validation_indices = random.sample(list(range(429)), int(0.1*429))
positive_1_train_indices = list(filter(lambda x: x not in positive_1_validation_indices, list(range(429))))
print(len(positive_1_train_indices), len(positive_1_validation_indices))

positive_2_imgs = glob.glob(os.path.join(POSITIVE_1_IMGS_FOLDER,'*'))
positive_2_validation_indices = random.sample(list(range(429, 1600)), int(0.1*1169))
positive_2_train_indices = list(filter(lambda x: x not in positive_2_validation_indices, list(range(429, 1600))))
print(len(positive_2_train_indices), len(positive_2_validation_indices))

negative_imgs = glob.glob(os.path.join(POSITIVE_1_IMGS_FOLDER,'*'))
negative_validation_indices = random.sample(list(range(429)), int(0.1*2084))
negative_train_indices = list(filter(lambda x: x not in negative_validation_indices, list(range(2084))))
print(len(negative_train_indices), len(negative_validation_indices))

#TODO: FIVE FOLD CV

# clear folder if already exsits
# folder_collections = [POSITIVE_1_IMGS_TRAIN_FOLDER, \
#                        POSITIVE_1_IMGS_VALIDATION_FOLDER, \
#                        POSITIVE_2_IMGS_TRAIN_FOLDER,
#                        POSITIVE_2_IMGS_VALIDATION_FOLDER, \
#                        NEGATIVE_IMGS_TRAIN_FOLDER, \
#                        NEGATIVE_IMGS_VALIDATION_FOLDER]
# for current_folder in folder_collections:
#     shutil.rmtree(current_folder)
#     os.makedirs(current_folder)

#save and copy
def copy_train_validation_image(image_index, source_image_folder, des_image_folder):
    source_path = os.path.join(source_image_folder, str(image_index) + '.jpg')
    paste_path = os.path.join(des_image_folder, str(image_index) + '.jpg')
#     print(image)
    shutil.copyfile(source_path, paste_path)
    
for i in positive_1_train_indices:
    copy_train_validation_image(i, POSITIVE_1_IMGS_FOLDER, POSITIVE_1_IMGS_TRAIN_FOLDER)
for i in positive_1_validation_indices:
    copy_train_validation_image(i, POSITIVE_1_IMGS_FOLDER, POSITIVE_1_IMGS_VALIDATION_FOLDER)
for i in positive_2_train_indices:
    copy_train_validation_image(i, POSITIVE_2_IMGS_FOLDER, POSITIVE_2_IMGS_TRAIN_FOLDER)
for i in positive_2_validation_indices:
    copy_train_validation_image(i, POSITIVE_2_IMGS_FOLDER, POSITIVE_2_IMGS_VALIDATION_FOLDER)
for i in negative_train_indices:
    copy_train_validation_image(i, NEGATIVE_IMGS_FOLDER, NEGATIVE_IMGS_TRAIN_FOLDER)
for i in negative_validation_indices:
    copy_train_validation_image(i, NEGATIVE_IMGS_FOLDER, NEGATIVE_IMGS_VALIDATION_FOLDER)

387 42
1055 116
1876 208


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\wiqwe\\Desktop\\上交科研\\selected_positive_#1_imgs_train\\0.jpg'

# image preprocessing pipeline

In [38]:
import numpy as np
import PIL.Image
import glob, os, shutil
POSITIVE_1_IMGS_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_positive_#1_imgs'
POSITIVE_1_IMGS_TRAIN_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_positive_#1_imgs_train'
POSITIVE_1_IMGS_VALIDATION_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_positive_#1_imgs_validation'
POSITIVE_2_IMGS_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_positive_#2_imgs'
POSITIVE_2_IMGS_TRAIN_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_positive_#2_imgs_train'
POSITIVE_2_IMGS_VALIDATION_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_positive_#2_imgs_validation'
NEGATIVE_IMGS_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_negative_imgs'
NEGATIVE_IMGS_TRAIN_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_negative_imgs_train'
NEGATIVE_IMGS_VALIDATION_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_negative_imgs_validation'

train_data = []
train_label = []
validation_data = []
validation_label = []

folder_collections = [POSITIVE_1_IMGS_TRAIN_FOLDER, \
                       POSITIVE_1_IMGS_VALIDATION_FOLDER, \
                       POSITIVE_2_IMGS_TRAIN_FOLDER,
                       POSITIVE_2_IMGS_VALIDATION_FOLDER, \
                       NEGATIVE_IMGS_TRAIN_FOLDER, \
                       NEGATIVE_IMGS_VALIDATION_FOLDER]
label_collections = [[0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [1, 0, 0]]
    
for folder_index, current_folder in enumerate(folder_collections):
    for img in glob.glob(os.path.join(current_folder ,'*')):
        im = PIL.Image.open(img)
        # im = im.convert('LA')
        im = im.resize((224,224))
        im = np.array(im)
        im = (im / 255.0) - 1.0
    #     im_roll = np.rollaxis(im, 2, 0)
    #     assert (im[:,:,0] == im_roll[0,:,:]).all()
    #     assert (im[:,:,1] == im_roll[1,:,:]).all()
    #     assert (im[:,:,2] == im_roll[2,:,:]).all()
        if 'train' in current_folder:
            train_data.append(np.array(im))
            train_label.append(label_collections[folder_index])
        elif 'validation' in current_folder:
            validation_data.append(np.array(im))
            validation_label.append(label_collections[folder_index])
        else:
            print(current_folder)
            raise TypeError

In [20]:
print(len(train_data), train_data[0].shape, len(train_label))
print(len(validation_data), validation_data[0].shape, len(validation_label))

3318 (224, 224, 3) 3318
366 (224, 224, 3) 366


In [ ]:
recovered_img = ((train_data[0]+1)*255).astype(np.uint8)
print(recovered_img.shape)
PIL.Image.fromarray(recovered_img).show()

In [27]:
train_data = np.stack(train_data, axis=0)
train_label = np.array(train_label)
validation_data = np.stack(validation_data, axis=0)
validation_label = np.array(validation_label)
print(train_data.shape, train_label.shape, validation_data.shape, validation_label.shape)

(3318, 224, 224, 3) (3318, 3) (366, 224, 224, 3) (366, 3)


# Train Resnet 50

In [33]:
import tensorflow as tf
import keras
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
keras.backend.set_session(sess)

In [34]:
resnet_50 = ResnetBuilder.build_resnet_50((3,224,224), 3)

In [35]:
optimizer = keras.optimizers.Adam(lr=0.001, amsgrad=True, decay=0.00005)
resnet_50.compile(optimizer, loss=keras.losses.categorical_crossentropy, metrics=['categorical_accuracy'])

In [ ]:
# callback function to save model every 10 epoch

In [ ]:
# learning rate schedular

In [ ]:
# Parameter searching for resnet 50, train for 100 epoch
# lr -> {0.001, 0.005, 0.0005}
# decay -> {0.00001, 0.0001, 0.0005}
# l2 - > {1e-3}

In [36]:
resnet_50.fit(train_data, train_label, epochs=50, batch_size=32, validation_data=(validation_data, validation_label))

Train on 3318 samples, validate on 366 samples
Epoch 1/50
3318/3318 [==============================] - 468s 141ms/step - loss: 25.1410 - categorical_accuracy: 0.8382 - val_loss: 10.8607 - val_categorical_accuracy: 0.5683
Epoch 2/50
3318/3318 [==============================] - 45s 14ms/step - loss: 5.9603 - categorical_accuracy: 0.8927 - val_loss: 4.1703 - val_categorical_accuracy: 0.7459
Epoch 3/50
3318/3318 [==============================] - 45s 14ms/step - loss: 2.8307 - categorical_accuracy: 0.9075 - val_loss: 3.5253 - val_categorical_accuracy: 0.3880
Epoch 4/50
3318/3318 [==============================] - 45s 14ms/step - loss: 1.9418 - categorical_accuracy: 0.9147 - val_loss: 2.1867 - val_categorical_accuracy: 0.6038
Epoch 5/50
3318/3318 [==============================] - 45s 14ms/step - loss: 1.4978 - categorical_accuracy: 0.9283 - val_loss: 1.7504 - val_categorical_accuracy: 0.6639
Epoch 6/50
3318/3318 [==============================] - 45s 14ms/step - loss: 1.2413 - categorical_

KeyboardInterrupt: 

# Evaluation

# keras Resnet definition

In [32]:
from __future__ import division

import six

from keras.models import Model

from keras.layers import (

    Input,

    Activation,

    Dense,

    Flatten

)

from keras.layers.convolutional import (

    Conv2D,

    MaxPooling2D,

    AveragePooling2D

)

from keras.layers.merge import add

from keras.layers.normalization import BatchNormalization

from keras.regularizers import l2

from keras import backend as K





def _bn_relu(input):

    """Helper to build a BN -> relu block

    """

    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)

    return Activation("relu")(norm)





def _conv_bn_relu(**conv_params):

    """Helper to build a conv -> BN -> relu block

    """

    filters = conv_params["filters"]

    kernel_size = conv_params["kernel_size"]

    strides = conv_params.setdefault("strides", (1, 1))

    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")

    padding = conv_params.setdefault("padding", "same")

    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-3))



    def f(input):

        conv = Conv2D(filters=filters, kernel_size=kernel_size,

                      strides=strides, padding=padding,

                      kernel_initializer=kernel_initializer,

                      kernel_regularizer=kernel_regularizer)(input)

        return _bn_relu(conv)



    return f





def _bn_relu_conv(**conv_params):

    """Helper to build a BN -> relu -> conv block.

    This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf

    """

    filters = conv_params["filters"]

    kernel_size = conv_params["kernel_size"]

    strides = conv_params.setdefault("strides", (1, 1))

    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")

    padding = conv_params.setdefault("padding", "same")

    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-3))



    def f(input):

        activation = _bn_relu(input)

        return Conv2D(filters=filters, kernel_size=kernel_size,

                      strides=strides, padding=padding,

                      kernel_initializer=kernel_initializer,

                      kernel_regularizer=kernel_regularizer)(activation)



    return f





def _shortcut(input, residual):

    """Adds a shortcut between input and residual block and merges them with "sum"

    """

    # Expand channels of shortcut to match residual.

    # Stride appropriately to match residual (width, height)

    # Should be int if network architecture is correctly configured.

    input_shape = K.int_shape(input)

    residual_shape = K.int_shape(residual)

    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))

    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))

    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]



    shortcut = input

    # 1 X 1 conv if shape is different. Else identity.

    if stride_width > 1 or stride_height > 1 or not equal_channels:

        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],

                          kernel_size=(1, 1),

                          strides=(stride_width, stride_height),

                          padding="valid",

                          kernel_initializer="he_normal",

                          kernel_regularizer=l2(0.001))(input)



    return add([shortcut, residual])





def _residual_block(block_function, filters, repetitions, is_first_layer=False):

    """Builds a residual block with repeating bottleneck blocks.

    """

    def f(input):

        for i in range(repetitions):

            init_strides = (1, 1)

            if i == 0 and not is_first_layer:

                init_strides = (2, 2)

            input = block_function(filters=filters, init_strides=init_strides,

                                   is_first_block_of_first_layer=(is_first_layer and i == 0))(input)

        return input



    return f





def basic_block(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):

    """Basic 3 X 3 convolution blocks for use on resnets with layers <= 34.

    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf

    """

    def f(input):



        if is_first_block_of_first_layer:

            # don't repeat bn->relu since we just did bn->relu->maxpool

            conv1 = Conv2D(filters=filters, kernel_size=(3, 3),

                           strides=init_strides,

                           padding="same",

                           kernel_initializer="he_normal",

                           kernel_regularizer=l2(1e-3))(input)

        else:

            conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3),

                                  strides=init_strides)(input)



        residual = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)

        return _shortcut(input, residual)



    return f





def bottleneck(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):

    """Bottleneck architecture for > 34 layer resnet.

    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf



    Returns:

        A final conv layer of filters * 4

    """

    def f(input):



        if is_first_block_of_first_layer:

            # don't repeat bn->relu since we just did bn->relu->maxpool

            conv_1_1 = Conv2D(filters=filters, kernel_size=(1, 1),

                              strides=init_strides,

                              padding="same",

                              kernel_initializer="he_normal",

                              kernel_regularizer=l2(1e-3))(input)

        else:

            conv_1_1 = _bn_relu_conv(filters=filters, kernel_size=(1, 1),

                                     strides=init_strides)(input)



        conv_3_3 = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv_1_1)

        residual = _bn_relu_conv(filters=filters * 4, kernel_size=(1, 1))(conv_3_3)

        return _shortcut(input, residual)



    return f





def _handle_dim_ordering():

    global ROW_AXIS

    global COL_AXIS

    global CHANNEL_AXIS

    if K.image_dim_ordering() == 'tf':

        ROW_AXIS = 1

        COL_AXIS = 2

        CHANNEL_AXIS = 3

    else:

        CHANNEL_AXIS = 1

        ROW_AXIS = 2

        COL_AXIS = 3





def _get_block(identifier):

    if isinstance(identifier, six.string_types):

        res = globals().get(identifier)

        if not res:

            raise ValueError('Invalid {}'.format(identifier))

        return res

    return identifier





class ResnetBuilder(object):

    @staticmethod

    def build(input_shape, num_outputs, block_fn, repetitions):

        """Builds a custom ResNet like architecture.



        Args:

            input_shape: The input shape in the form (nb_channels, nb_rows, nb_cols)

            num_outputs: The number of outputs at final softmax layer

            block_fn: The block function to use. This is either `basic_block` or `bottleneck`.

                The original paper used basic_block for layers < 50

            repetitions: Number of repetitions of various block units.

                At each block unit, the number of filters are doubled and the input size is halved



        Returns:

            The keras `Model`.

        """

        _handle_dim_ordering()

        if len(input_shape) != 3:

            raise Exception("Input shape should be a tuple (nb_channels, nb_rows, nb_cols)")



        # Permute dimension order if necessary

        if K.image_dim_ordering() == 'tf':

            input_shape = (input_shape[1], input_shape[2], input_shape[0])



        # Load function from str if needed.

        block_fn = _get_block(block_fn)



        input = Input(shape=input_shape)

        conv1 = _conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(input)

        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)



        block = pool1

        filters = 64

        for i, r in enumerate(repetitions):

            block = _residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i == 0))(block)

            filters *= 2



        # Last activation

        block = _bn_relu(block)



        # Classifier block

        block_shape = K.int_shape(block)

        pool2 = AveragePooling2D(pool_size=(block_shape[ROW_AXIS], block_shape[COL_AXIS]),

                                 strides=(1, 1))(block)

        flatten1 = Flatten()(pool2)

        dense = Dense(units=num_outputs, kernel_initializer="he_normal",

                      activation="softmax")(flatten1)



        model = Model(inputs=input, outputs=dense)

        return model



    @staticmethod

    def build_resnet_18(input_shape, num_outputs):

        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [2, 2, 2, 2])



    @staticmethod

    def build_resnet_34(input_shape, num_outputs):

        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [3, 4, 6, 3])



    @staticmethod

    def build_resnet_50(input_shape, num_outputs):

        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 6, 3])



    @staticmethod

    def build_resnet_101(input_shape, num_outputs):

        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 23, 3])



    @staticmethod

    def build_resnet_152(input_shape, num_outputs):

        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 8, 36, 3])